In [2]:
import casadi as ca
import sys
sys.path.insert(0, '../python/pyecca/')
from pyecca.lie import so3
from pyecca.util import rk4

from IPython.display import Image
from casadi.tools.graph import dotgraph

def draw_graph(expr):
    return Image(dotgraph(expr).create_png())

ModuleNotFoundError: No module named 'pyecca'

$ ^e\dfrac{d}{dt}(mv) = F$ Newton's 2nd law. Derivative of linear momentum with respect to the inertial frame is equal to the force applied on a rigid body.

$ ^e\dfrac{d}{dt}(J^e\omega^b) = M$ Newton's 2nd law (angular cersion). Derivative of the angular momentum with respect to the inertial frame is equal to the moment applied on the rigid body.

$\dot{m} v + m \dfrac{^ed}{dt} = A$

$\dot{m} v_e + m a_e = A$

In [4]:
def rocket_rhs():
    x = ca.SX.sym('x', 14)
    u = ca.SX.sym('u', 1)
    p = ca.SX.sym('p', 6)

    # State: x
    omega_b = x[0:3]  # inertial angular velocity expressed in body frame
    r_nb = x[3:7]  # modified rodrigues parameters
    v_b = x[7:10]  # body veloctiy
    p_n = x[10:13]  # positon in nav frame
    m = x[13]
    
    # Input: u    
    m_dot = u[0]
    
    # Parameters: p
    g = p[0]  # gravity
    Jx = p[1]  # moment of inertia
    Jy = p[2]
    Jz = p[3]
    Jxz = p[4]
    ve = p[5]

    # Calculations
    VT = ca.norm_2(v_b)
    F_b = ca.vertcat(m_dot*ve, 0, 0)
    M_b = ca.vertcat(0, 0, 0)
    J_b = ca.SX.zeros(3, 3)
    J_b[0, 0] = Jx
    J_b[1, 1] = Jy
    J_b[2, 2] = Jz
    J_b[0, 2] = J_b[2, 0] = Jxz
    C_nb = so3.Dcm.from_mrp(r_nb)
    g_n = ca.vertcat(0, 0, g)
    v_n = ca.mtimes(C_nb, v_b)
    
    return ca.Function('rhs', [x, u, p], [ca.vertcat(
        ca.mtimes(ca.inv(J_b), M_b - ca.cross(omega_b, ca.mtimes(J_b, omega_b))),
        so3.Mrp.kinematics(r_nb, omega_b),
        F_b/m - m_dot*v_b/m - ca.cross(omega_b, v_b),
        ca.mtimes(C_nb, v_b))], ['x', 'u', 'p'], ['rhs'])


p0 = ca.vertcat(9.8, 1, 1, 1, 0.1, 350)
# x: omega_b, q_nb, v_b, p_n, m
x0 = ca.vertcat(0, 0, 0, so3.Mrp.from_euler(ca.vertcat(0, 0, 0)), 0, 0, 0, 0, 0, 0, 1)
u0 = ca.vertcat(0.2)
rhs = rocket_rhs()

x = ca.SX.sym('x', 14)
u = ca.SX.sym('u', 1)
p = ca.SX.sym('p', 6)
rhs(x, u, p)

NameError: name 'so3' is not defined

In [ ]:
draw_graph(rhs(x, u, p))